## The code for merging all the data into one, cleaned table

In [69]:
import geopandas as gpd
import pandas as pd

### Defining function to load data

In [70]:
def get_data(path, columns_to_delete=[], geo=False, sep=','):
    '''
    Function to load the data using pandas and geopandas.
    Loads all geospacial data regardles of the file extention.
    Non-geospatial data is limited to CSV files with definable separators.
    Inputs:
        path: file path to the data file
        columns_to_delete: string or list with column names to drop (default: [])
        geo: bool to define whether the file contains geospatial data (default: False)
        sep: CSV separator (default: ',')
    Returns:
        A pandas.DataFrame object or a geopandas.GeoDataFrame
        depending on the geo parameter
    '''
    if geo:
        return (
            gpd.read_file(path)
                .drop(columns_to_delete, axis=1)
        )
    else:
        return (
            pd.read_csv(path, sep=sep)
                .drop(columns_to_delete, axis=1)
        )

### Loading all the data

In [71]:
df_geo = get_data('data_raw\\utils.geojson', [], True)
df_jobs = get_data('data_raw\jobs_by_buurt.csv')
df_green_neighborhood = get_data('data_raw\green_live_buurt_complete.csv')
df_price = get_data('data_raw\housing_prices.csv')
df_prox = get_data('data_raw\proximity.csv')
df_nuisance = get_data('data_raw\\nuisance.csv', sep=';')
df_crime = get_data('data_raw\crime_by_type.csv')
df_drugs = get_data('data_raw\drugscords.csv', 
                    ['totalScore',
                     'reviewsCount',
                     'street',
                     'city',
                     'state',
                     'countryCode',
                     'website',
                     'phone',
                     'categoryName',
                     'url'])

In [72]:
df_drugs = gpd.GeoDataFrame(df_drugs,
                            geometry=gpd.points_from_xy(df_drugs['longitude'], df_drugs['latitude'], crs="EPSG:4326"))
df_drugs = df_drugs.drop(['latitude', 'longitude'], axis=1)

In [73]:
df_drugs.head()

,title,geometry
0,the Baron,POINT (4.78469 51.59150)
1,Pax,POINT (4.76606 51.59057)
2,Coffeeshop Paradise,POINT (4.77811 51.58868)
3,Sky Coffeeshop,POINT (4.76005 51.58442)
4,The Cat,POINT (4.80523 51.58700)


In [74]:
df_prox = df_prox[df_prox['within'] == 'Within 3 km'].drop(['Unnamed: 0', 'within'], axis=1)
df_prox = df_prox.groupby(['zones', 'catetgory'], as_index=False)['Value'].sum()
df_prox = df_prox.pivot_table(index='zones', columns='catetgory', values='Value')

### Looking at the columns available

In [75]:
df_geo.head(1)

,regions,neighborhood,light_count,sport_building_count,workplace_count,inhabitants,light_per_1000,sport_building_per_1000,area_sqkm,distance_from_centre_km,geometry
0,Bavel,Bavel,1253,2,2,5445,230.119376,0.367309,1.750374,4.59422,"POLYGON ((4.83844 51.56120, 4.83911 51.56024, ..."


In [76]:
df_jobs.head(1)

,Buurt,Value
0,Chassé,1920


In [77]:
df_green_neighborhood.head(1)

,Unnamed: 0,BUURT,green_score,livability_score
0,0,Bavel,30.375317,2.35557


In [78]:
df_price.head(1)

,BUURT,price
0,Belcrum,4231


In [79]:
df_prox.head(1)

catetgory,Childcare,Education,Health and well-being,Hospitality,Retail
zones,,,,,
Bavel,11.3,2.6,3.5,10.7,15.8


### Merging the tables

In [80]:
df_join = (
    df_geo.merge(df_green_neighborhood, how='left', left_on='neighborhood', right_on='BUURT')
        .drop(['BUURT', 'Unnamed: 0'], axis=1)
)

In [81]:
df_join.shape

(56, 13)

In [82]:
df_join = (
    df_join.merge(df_price, how='left', left_on='neighborhood', right_on='BUURT')
        .drop('BUURT', axis=1)
        .rename(columns={'price': 'house_price_per_m'})
)

In [83]:
df_join.columns

Index(['regions', 'neighborhood', 'light_count', 'sport_building_count',
       'workplace_count', 'inhabitants', 'light_per_1000',
       'sport_building_per_1000', 'area_sqkm', 'distance_from_centre_km',
       'geometry', 'green_score', 'livability_score', 'house_price'],
      dtype='object')

In [84]:
df_join.shape

(56, 14)

In [48]:
df_join = df_join.sjoin(df_drugs, how='left')

(158, 15)

In [50]:
df_join = df_join.dissolve(by=['regions', 'neighborhood', 'light_count', 'sport_building_count',
       'workplace_count', 'inhabitants', 'light_per_1000',
       'sport_building_per_1000', 'area_sqkm', 'distance_from_centre_km',
        'green_score', 'livability_score'],
                                aggfunc='count',
                                as_index=False)
df_join = df_join.drop('index_right', axis=1)
df_join = df_join.rename(columns={'title': 'drug_store_count'})

In [51]:
df_join.shape

(53, 14)

In [52]:
df_join.explore()

In [53]:
df_join.head()

,regions,neighborhood,light_count,sport_building_count,workplace_count,inhabitants,light_per_1000,sport_building_per_1000,area_sqkm,distance_from_centre_km,green_score,livability_score,geometry,drug_store_count
0,Bavel,Bavel,1253,2,2,5445,230.119376,0.367309,1.750374,4.594220,30.375317,2.355570,"POLYGON ((4.83844 51.56120, 4.83911 51.56024, ...",0
1,Bavel,Buitengebied Bavel,308,6,1,570,540.350877,10.526316,8.559273,5.384812,29.375000,2.499306,"POLYGON ((4.87112 51.56696, 4.87096 51.56694, ...",0
2,Bavel,Nieuw Wolfslaar,515,0,0,2365,217.758985,0.000000,1.027985,3.831573,25.379099,2.528201,"POLYGON ((4.81440 51.55980, 4.81426 51.55978, ...",0
3,Breda centrum,Chassé,647,0,3,3365,192.273403,0.000000,0.408947,0.566532,15.346780,2.262315,"POLYGON ((4.78152 51.58244, 4.78139 51.58235, ...",5
4,Breda centrum,City,668,0,3,2575,259.417476,0.000000,0.435064,0.000000,9.984778,2.169985,"POLYGON ((4.77889 51.58641, 4.77902 51.58572, ...",47


In [55]:
df_join.shape

(53, 15)

In [56]:
df_join.head(1)

,regions,neighborhood,light_count,sport_building_count,workplace_count,inhabitants,light_per_1000,sport_building_per_1000,area_sqkm,distance_from_centre_km,green_score,livability_score,geometry,drug_store_count,house_price
0,Bavel,Bavel,1253,2,2,5445,230.119376,0.367309,1.750374,4.59422,30.375317,2.35557,"POLYGON ((4.83844 51.56120, 4.83911 51.56024, ...",0,4370.0


In [57]:
df_join = (
    df_join.merge(df_jobs, how='inner', left_on='neighborhood', right_on='Buurt')
        .drop('Buurt', axis=1)
        .rename(columns={'Value': 'jobs_count'})
)

In [58]:
df_join = df_join.merge(df_nuisance, how='left', left_on='neighborhood', right_on='Districts and neighbourhoods')

In [59]:
df_join = df_join.merge(df_crime, how='inner', on='neighborhood')

In [60]:
df_join.head(1)

,regions,neighborhood,light_count,sport_building_count,workplace_count,inhabitants,light_per_1000,sport_building_per_1000,area_sqkm,distance_from_centre_km,...,Theft/burglary box/garage/shed,"Theft/burglary of companies, etc.",Thefts (water),Threat,Total felonies,Under the influence (air),Under the influence (away),Vertical Fraud,Waste,Water
0,Bavel,Bavel,1253,2,2,5445,230.119376,0.367309,1.750374,4.59422,...,4,5,0,16,366,0,12,0,0,0


In [61]:
df_join = df_join.merge(df_prox, how='left', left_on='neighborhood', right_on='zones')

### Reorganising the columns (moving geometry column to the end), getting rid of broken columns

In [62]:
df_join.columns

Index(['regions', 'neighborhood', 'light_count', 'sport_building_count',
       'workplace_count', 'inhabitants', 'light_per_1000',
       'sport_building_per_1000', 'area_sqkm', 'distance_from_centre_km',
       'green_score', 'livability_score', 'geometry', 'drug_store_count',
       'house_price', 'jobs_count', 'Districts and neighbourhoods',
       'Total nuisance registrations', 'Nuisance by confused person',
       'Youth nuisance report', 'Nuisance due to alcohol/drugs',
       'Nuisance drifters', 'Public intoxication', 'Noise nuisance catering',
       'Noise nuisance event', 'Other noise nuisance', 'Unnamed: 0',
       'Accidents (road)', 'Encroachment on public order', 'Fraud (other)',
       'Horizontal Fraud', 'Human trafficking', 'Nature and landscape',
       'Quality of life (other)', 'Road (other)', 'Spatial planning',
       'Special Laws', 'Transport of hazardous substances',
       'Under the influence (water)', 'Abuse', 'Air (other)', 'Animals',
       'Arms Trade'

In [63]:
df_join = df_join[['regions', 'neighborhood', 'light_count', 'sport_building_count',
       'workplace_count', 'inhabitants', 'light_per_1000',
       'sport_building_per_1000', 'area_sqkm', 'distance_from_centre_km',
       'green_score', 'livability_score', 'geometry', 'drug_store_count',
       'house_price', 'jobs_count', 'Districts and neighbourhoods',
       'Total nuisance registrations', 'Nuisance by confused person',
       'Youth nuisance report', 'Nuisance due to alcohol/drugs',
       'Nuisance drifters', 'Public intoxication', 'Noise nuisance catering',
       'Noise nuisance event', 'Other noise nuisance',
       'Accidents (road)', 'Encroachment on public order', 'Fraud (other)',
       'Horizontal Fraud', 'Human trafficking', 'Nature and landscape',
       'Quality of life (other)', 'Road (other)', 'Spatial planning',
       'Special Laws', 'Transport of hazardous substances',
       'Under the influence (water)', 'Abuse', 'Air (other)', 'Animals',
       'Arms Trade', 'Building materials', 'Cybercrime',
       'Discrimination',
       'Domestic Violation', 'Drug trafficking', 'Drugs/drink nuisance',
       'Fire/Explosion', 'Fireworks', 'Food safety', 'Home theft/burglary',
       'Immigration care', 'Most', 'Motor Vehicle Theft',
       'Murder, Manslaughter', 'Neighbor rumor (relationship problems)',
       'Open violence (person)', 'Other property crimes', 'People smuggling',
       'Pesticides', 'Pickpocketing', 'Robbery', 'Shoplifting', 'Soil',
       'Street robbery', 'Structure of the Environmental Management Act',
       'Theft from/from motor vehicles',
       'Theft of mopeds, mopeds and bicycles',
       'Theft/burglary box/garage/shed', 'Theft/burglary of companies, etc.',
       'Thefts (water)', 'Threat', 'Total felonies',
       'Under the influence (air)', 'Under the influence (away)',
       'Vertical Fraud', 'Waste', 'Water', 'Childcare', 'Education',
       'Health and well-being', 'Hospitality', 'Retail']]

In [64]:
df_join['proximity_score'] = (df_join['Childcare']+df_join['Education']+df_join['Health and well-being']
                                +df_join['Hospitality']+df_join['Retail']) / 5

In [65]:
df_join = df_join.drop('Districts and neighbourhoods', axis=1)
df_join = df_join.rename(columns={'Under the influence (away)': 'Under the influence (road)'})

In [66]:
df_join

,regions,neighborhood,light_count,sport_building_count,workplace_count,inhabitants,light_per_1000,sport_building_per_1000,area_sqkm,distance_from_centre_km,...,Under the influence (road),Vertical Fraud,Waste,Water,Childcare,Education,Health and well-being,Hospitality,Retail,proximity_score
0,Bavel,Bavel,1253,2,2,5445,230.119376,0.367309,1.750374,4.594220,...,12,0,0,0,11.3,2.6,3.5,10.7,15.8,8.78
1,Bavel,Buitengebied Bavel,308,6,1,570,540.350877,10.526316,8.559273,5.384812,...,11,0,0,0,12.6,3.3,3.1,13.1,15.2,9.46
2,Bavel,Nieuw Wolfslaar,515,0,0,2365,217.758985,0.000000,1.027985,3.831573,...,1,0,0,0,34.3,6.1,7.4,40.5,36.4,24.94
3,Breda centrum,Chassé,647,0,3,3365,192.273403,0.000000,0.408947,0.566532,...,27,1,0,0,83.3,16.2,17.5,423.9,156.3,139.44
4,Breda centrum,City,668,0,3,2575,259.417476,0.000000,0.435064,0.000000,...,22,1,0,0,80.0,17.5,16.4,420.5,152.3,137.34
5,Breda centrum,Fellenoord,240,0,2,1565,153.354633,0.000000,0.277823,0.648226,...,12,0,0,0,79.2,16.7,15.5,424.4,148.6,136.88
6,Breda centrum,Schorsmolen,329,0,1,3450,95.362319,0.000000,0.276399,0.584711,...,30,0,0,0,68.5,15.8,15.0,415.9,141.6,131.36
7,Breda centrum,Station,318,0,3,2310,137.662338,0.000000,0.291627,0.692898,...,22,0,0,0,62.9,16.7,14.4,392.7,141.4,125.62
8,Breda centrum,Valkenberg,402,0,3,2005,200.498753,0.000000,0.414723,0.594466,...,13,0,0,0,75.1,19.1,17.7,408.9,154.3,135.02
9,Breda noord,Belcrum,637,0,4,4005,159.051186,0.000000,0.532119,0.935204,...,12,0,0,0,54.0,12.7,12.1,374.6,131.2,116.92


### Saving the merged data to a GeoJSON

In [67]:
df_join.head(1)

,regions,neighborhood,light_count,sport_building_count,workplace_count,inhabitants,light_per_1000,sport_building_per_1000,area_sqkm,distance_from_centre_km,...,Under the influence (road),Vertical Fraud,Waste,Water,Childcare,Education,Health and well-being,Hospitality,Retail,proximity_score
0,Bavel,Bavel,1253,2,2,5445,230.119376,0.367309,1.750374,4.59422,...,12,0,0,0,11.3,2.6,3.5,10.7,15.8,8.78


In [68]:
df_join.to_file("data_merged\\full_join.geojson", driver='GeoJSON')